In [1]:
# import the dependencies
import pandas as pd
import gmaps
import requests
%matplotlib inline
import ipywidgets as widgets
# import api key
from config import g_key
import json

In [2]:
# store the csv into df
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bardiyah,LY,2021-10-25 21:56:04,31.7561,25.0865,70.29,75,67,14.29
1,1,Mbandaka,CD,2021-10-25 21:56:04,0.0487,18.2603,71.53,87,77,3.18
2,2,Albany,US,2021-10-25 21:54:16,42.6001,-73.9662,55.94,90,100,3.27
3,3,Chokurdakh,RU,2021-10-25 21:56:04,70.6333,147.9167,-23.42,100,97,1.77
4,4,Motygino,RU,2021-10-25 21:56:05,58.1831,94.7592,40.80,75,98,15.86


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# configures maps to use Google api key
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# heatmap of temperature
# get the latitude and longitude
locations = city_data_df[["Lat","Lng"]]

# get the maximum temperature
max_temp = city_data_df["Max Temp"]

#assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False, max_intensity=300, point_radius=4)

# add the heatmap layer
fig.add_layer(heat_layer)

# call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# create heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False,
                                max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# create wind speed heatmap
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False,
                                max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# ask the customer to add a min and max temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [11]:
# filter dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)
                                      & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Cururupu,BR,2021-10-25 21:56:06,-1.8283,-44.8683,79.61,79,38,5.79
12,12,Victoria,HK,2021-10-25 21:53:59,22.2855,114.1577,75.33,76,99,1.99
15,15,Port Keats,AU,2021-10-25 21:56:09,-14.2500,129.5500,84.02,68,14,1.57
18,18,Urucara,BR,2021-10-25 21:56:09,-2.5364,-57.7600,82.35,65,74,2.35
23,23,Cabo San Lucas,MX,2021-10-25 21:55:55,22.8909,-109.9124,89.51,66,0,12.15
27,27,Angoche,MZ,2021-10-25 21:56:13,-16.2325,39.9086,77.74,90,0,5.95
29,29,Touros,BR,2021-10-25 21:54:49,-5.1989,-35.4608,81.50,72,1,16.44
33,33,Bafoulabe,ML,2021-10-25 21:56:14,13.8065,-10.8321,79.39,73,100,3.00
37,37,Ambulu,ID,2021-10-25 21:56:16,-8.3450,113.6058,76.42,84,93,1.16
38,38,Vaini,TO,2021-10-25 21:56:16,-21.2000,-175.2000,77.16,83,20,9.22


In [12]:
preferred_cities_df.count()

City_ID       169
City          169
Country       169
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [13]:
# create df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Cururupu,BR,79.61,-1.8283,-44.8683,
12,Victoria,HK,75.33,22.2855,114.1577,
15,Port Keats,AU,84.02,-14.2500,129.5500,
18,Urucara,BR,82.35,-2.5364,-57.7600,
23,Cabo San Lucas,MX,89.51,22.8909,-109.9124,
27,Angoche,MZ,77.74,-16.2325,39.9086,
29,Touros,BR,81.50,-5.1989,-35.4608,
33,Bafoulabe,ML,79.39,13.8065,-10.8321,
37,Ambulu,ID,76.42,-8.3450,113.6058,
38,Vaini,TO,77.16,-21.2000,-175.2000,


In [21]:
# set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Cururupu,BR,79.61,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
12,Victoria,HK,75.33,22.2855,114.1577,Mini Hotel Central
15,Port Keats,AU,84.02,-14.2500,129.5500,TDC Village (Hundred Man Camp)
18,Urucara,BR,82.35,-2.5364,-57.7600,Pousada Maria Bonita
23,Cabo San Lucas,MX,89.51,22.8909,-109.9124,Hotel Tesoro Los Cabos
27,Angoche,MZ,77.74,-16.2325,39.9086,Guest House HESADA
29,Touros,BR,81.50,-5.1989,-35.4608,INN NEW HORIZON
33,Bafoulabe,ML,79.39,13.8065,-10.8321,
37,Ambulu,ID,76.42,-8.3450,113.6058,Hotel Ambulu Jember
38,Vaini,TO,77.16,-21.2000,-175.2000,Keleti Beach Resort


In [23]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [25]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))